# Some quick start code for TUM Hackathon

In [ ]:
from langchain.document_loaders import PyPDFLoader
from ai_eval.config import global_config as glob

filename = "Allplan_2020_Manual.pdf"

loader = PyPDFLoader(f"{glob.DATA_PKG_DIR}/{filename}")

raw_data = loader.load()

texts = [page.page_content for page in raw_data]

print(f"Number of docs: {len(texts)}")

/Users/avosseler/Github/Team/tum-hackathon/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of docs: 319


## (Optional) Preprocess and load data: 

In [ ]:
from ai_eval.resources.preprocessor import Preprocessor
from ai_eval.config import global_config as glob

filename = "Allplan_2020_Manual.pdf"

pre = Preprocessor()

docs = pre.fetch_documents(
    blob_path=f"{glob.DATA_PKG_DIR}/{filename}", source="local"
)

documents = pre.chunk_documents(documents=docs)

print(f"Number of processed document chunks: {len(documents)}")

2025-11-20 21:12:14,506 - ai_eval.resources.preprocessor - INFO - Processed /Users/avosseler/Github/Team/tum-hackathon/data/Allplan_2020_Manual.pdf
2025-11-20 21:12:14,514 - ai_eval.resources.preprocessor - INFO - Using SentenceChunker for document chunking.


Finished 'fetch_documents' in 1.9646 secs


🦛 choooooooooooooooooooonk 100% • 319/319 docs chunked [00:02<00:00, 133.13doc/s] 🌱
2025-11-20 21:12:17,351 - ai_eval.resources.preprocessor - INFO - Chunked 319 pages into 319 chunks.


Finished 'chunk_documents' in 2.8480 secs
Number of processed document chunks: 319


## Get annotated data:

In [ ]:
from ai_eval.services.file import JSONService
from ai_eval.config import global_config as glob

json = JSONService(path="generated_qa_data_tum.json", root_path=glob.DATA_PKG_DIR, verbose=True)

qa_data = json.doRead()
print(f"Number of evaluation data samples: {len(qa_data)}")

2025-11-20 21:36:31,914 - ai_eval.services.file - INFO - Read: /Users/avosseler/Github/Team/tum-hackathon/data/generated_qa_data_tum.json


Number of evaluation data samples: 100


### Fit RAG model on the generated data and create evaluation dataset

In [ ]:
from ai_eval.resources import eval_dataset_builder as eval

ground_truth_contexts = [item["context"] for item in qa_data]
sample_queries = [item["question"] for item in qa_data]
expected_responses = [item["answer"] for item in qa_data]

## Alternative: Using RAGPrototype (Qdrant + Jina + Claude)


In [ ]:
# Import RAGPrototype and required components
from ai_eval.resources.rag_prototype import (
    RAGPrototype,
    RAGConfig,
    RetrievalMode,
    EmbeddingModel,
    LLMModel,
)
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify required API keys are set
required_keys = {
    "QDRANT_URL": os.getenv("QDRANT_URL"),
    "QDRANT_API_KEY": os.getenv("QDRANT_API_KEY"),
    "JINA_API_KEY": os.getenv("JINA_API_KEY"),
    "ANTHROPIC_API_KEY": os.getenv("ANTHROPIC_API_KEY"),
}

missing_keys = [key for key, value in required_keys.items() if not value]
if missing_keys:
    raise ValueError(
        f"❌ Missing required API keys: {', '.join(missing_keys)}. "
        "Please set them in your .env file."
    )

print("✅ All required API keys are set")


In [ ]:
# Configure RAGPrototype
# You can customize these settings based on your needs
config = RAGConfig(
    # Collection settings
    collection_name="allplan_documents",
    force_recreate=False,  # Set to True to recreate the collection
    
    # Retrieval mode: TEXT_ONLY, MULTIMODAL, or HYBRID
    retrieval_mode=RetrievalMode.TEXT_ONLY,
    
    # Embedding settings
    embedding_model=EmbeddingModel.JINA_V4,  # Jina v4 supports multimodal
    embedding_dim=2048,
    
    # LLM settings
    llm_model=LLMModel.CLAUDE_HAIKU_4_5,  # Fast and cost-effective
    temperature=0.1,
    
    # Retrieval settings
    top_k=3,
    chunk_size=1024,
    chunk_overlap=200,
    
    # Vision settings (for multimodal mode)
    enable_vision=False,  # Set to True for multimodal
)

# Create RAGPrototype instance
rag_prototype = RAGPrototype(config=config, documents=documents)

print(f"✅ RAGPrototype initialized")
print(f"   Mode: {config.retrieval_mode.value}")
print(f"   Collection: {config.collection_name}")
print(f"   Embedding: {config.embedding_model.value}")
print(f"   LLM: {config.llm_model.value}")


In [ ]:
# Deploy embeddings to Qdrant
# This will create embeddings and store them in the vector database
deployment_metadata = rag_prototype.deploy_embeddings(
    documents=documents,
    force_redeploy=False  # Set to True to force redeployment
)

print(f"\n✅ Embeddings deployed successfully!")
print(f"   Documents: {deployment_metadata.num_documents}")
print(f"   Nodes: {deployment_metadata.num_nodes}")
print(f"   Duration: {deployment_metadata.duration_seconds:.2f}s")
print(f"   Collection: {deployment_metadata.collection_name}")

# Get collection info
collection_info = rag_prototype.get_collection_info()
print(f"\n📊 Collection Info:")
for key, value in collection_info.items():
    print(f"   {key}: {value}")


In [ ]:
# Test the RAGPrototype
query = "What is Allplan?"

# Retrieve relevant documents
the_relevant_docs = rag_prototype.retrieve(question=query)
print(f"Retrieved {len(the_relevant_docs)} relevant documents")

# Get full answer with context
answer, relevant_docs = rag_prototype.answer(question=query)

print(f"\n❓ Question: {query}")
print(f"\n💬 Answer:\n{answer}")
print(f"\n📄 Retrieved {len(relevant_docs)} documents")

# Show retrieval with scores (optional)
retrieval_result = rag_prototype.retrieve_structured(question=query, return_scores=True)
print(f"\n📊 Retrieval Stats:")
print(f"   Total retrieved: {retrieval_result.num_retrieved}")
print(f"   Text docs: {retrieval_result.num_text_docs}")
print(f"   Image docs: {retrieval_result.num_image_docs}")
if retrieval_result.avg_score:
    print(f"   Avg similarity: {retrieval_result.avg_score:.4f}")


### Using RAGPrototype with Evaluation Pipeline


In [ ]:
# Use RAGPrototype with the evaluation pipeline
# RAGPrototype inherits from RAG base class, so it's compatible with EvalDatasetBuilder

# Create the builder with RAGPrototype instance
builder_prototype = eval.EvalDatasetBuilder(rag_prototype)

# Build the evaluation dataset
evaluation_dataset_prototype = builder_prototype.build_evaluation_dataset(
    input_contexts=ground_truth_contexts,
    sample_queries=sample_queries,
    expected_responses=expected_responses,
)

print("✅ Evaluation dataset created with RAGPrototype")


In [ ]:
# Score the results using DeepEval
from ai_eval.resources import deepeval_scorer as deep

scorer_prototype = deep.DeepEvalScorer(evaluation_dataset_prototype)

results_prototype = scorer_prototype.calculate_scores()
print("Evaluation Results:")
print(results_prototype)


In [ ]:
# Get overall metrics
scorer_prototype.get_overall_metrics()


In [ ]:
# Get summary and save to file
scorer_prototype.get_summary(save_to_file=True)


### Optional: Switch to Multimodal Mode

If you want to enable multimodal capabilities (images, diagrams, tables):


In [ ]:
# Switch to multimodal mode (uncomment to use)
# This enables vision capabilities for processing images, diagrams, and tables

from ai_eval.resources.rag_prototype import ParseMode

# rag_prototype.switch_to_multimodal(
#     enable_vision=True,
#     parse_mode=ParseMode.MULTIMODAL_LVM  # Requires LlamaParse API key
# )

# Note: Multimodal mode requires:
# 1. enable_vision=True in config
# 2. Optional: LLAMAPARSE_API_KEY for advanced parsing
# 3. Documents with images/diagrams for best results

print("💡 To enable multimodal mode, uncomment the code above")


Example: using Vertex AI models

In [ ]:
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings

chat_model = ChatVertexAI(
                        project=glob.GCP_PROJECT,
                        model_name="gemini-2.5-flash",
                        temperature=0.1,
                        max_retries=2,
                    )

embedding_model = VertexAIEmbeddings(
                        project=glob.GCP_PROJECT,
                        model_name="text-embedding-005",
                    )

/Users/avosseler/Github/Team/tum-hackathon/.venv/lib/python3.13/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils
/Users/avosseler/Github/Team/tum-hackathon/.venv/lib/python3.13/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [ ]:
from langchain_community.vectorstores import FAISS
from ai_eval.resources.rag_template import FAISSRAG

vectorstore = FAISS.from_documents(documents, embedding_model)

# 1. Create your RAG instance
rag = FAISSRAG(chat_model, documents, k=3, vectorstore=vectorstore)    # some vanilla example
# rag = TFIDFRAG(models.qa_generator, documents, k=3)                 # our (naive) hackathon baseline 

query = "What is Allplan?"

the_relevant_docs = rag.retrieve(question=query)

answer, relevant_docs = rag.answer(question=query)

Building test dataset: 100%|██████████| 100/100 [13:22<00:00,  8.02s/it]
2025-11-20 21:26:08,453 - ai_eval.resources.eval_dataset_builder - INFO - Evaluation dataset created successfully.


In [ ]:
# 2. Create the builder with the RAG instance
builder = eval.EvalDatasetBuilder(rag)

# 3. Build the evaluation dataset
evaluation_dataset = builder.build_evaluation_dataset(
    input_contexts=ground_truth_contexts,
    sample_queries=sample_queries,
    expected_responses=expected_responses,
)

In [ ]:
from ai_eval.resources import deepeval_scorer as deep 

scorer = deep.DeepEvalScorer(evaluation_dataset)

results = scorer.calculate_scores()
print(results)

/Users/avosseler/Github/Team/tum-hackathon/.venv/lib/python3.13/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
2025-11-20 17:59:09,975 - ai_eval.resources.get_models - INFO - Using chat model: gemini-2.5-flash
2025-11-20 17:59:09,977 - ai_eval.resources.get_models - INFO - Using judge model: gemini-2.5-flash
2025-11-20 17:59:09,977 - ai_eval.resources.get_models - INFO - Using embedding model: text-embedding-005
2025-11-20 17:59:09,977 - ai_eval.resources.get_models - INFO - Models loaded successfully
2025-11-20 17:59:09,978 - ai_eval.resources.deepeval_scorer - INFO - Starting evaluation...
2025-11-20 17:59:09,979 - ai_eval.resources.deepeval_scorer - ERROR - Evaluation failed: If using 'test_cases', you must also provide 'met

Attempt 1 failed: If using 'test_cases', you must also provide 'metrics'..
Retrying...
Attempt 2 failed: If using 'test_cases', you must also provide 'metrics'..
Retrying...
Attempt 3 failed: If using 'test_cases', you must also provide 'metrics'..
Retrying...
{}


In [ ]:
scorer.get_overall_metrics()

{'Answer Relevancy': 0.9149721667221666,
 'Faithfulness': 0.9780194805194804,
 'Contextual Recall': 0.92,
 'Contextual Precision': 0.8933333333333333,
 'Average Performance': 0.9265812451437451}

In [ ]:
scorer.get_summary(save_to_file=True)

2025-11-20 01:05:10,490 - ai_eval.resources.deepeval_scorer - INFO - Summary of test results generated successfully!
2025-11-20 01:05:10,503 - ai_eval.services.file - INFO - JSON Service Output to File: /Users/avosseler/Github/Team/tum-hackathon/data/deepeval_results.json


{0: {'query': 'How do I sort documents in ascending or descending order in the software?',
  'actual_output': 'You can sort documents by clicking the title of a column.\n\n*   To sort documents in ascending order, click the column title.\n*   To sort documents in descending order, click the same column title again.\n\nAn arrow indicates which column is being sorted and whether sorting is in ascending (arrow points upward) or descending (arrow points downward) order.',
  'expected_output': 'You can sort documents by clicking the title of a column. Clicking it once sorts in ascending order, and clicking the same title again sorts in descending order. An arrow indicates the sorted column and direction.',
  'actual_context': ['Installation, Basics Structuring and managing data 251 Sorting documents You can sort the documents by clicking the title of a column. Click the column title to sort the documents in ascending order. Click the same column title again to sort the documents in descendi

I0000 00:00:1763597131.843614 4491286 ssl_transport_security.cc:1884] Handshake failed with error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED: unable to get local issuer certificate
